In [1]:
import os
os.chdir('..')
import sqlite3
from dotenv import load_dotenv
from selenium import webdriver
from helper import set_filters_huurstunt
from helper_database import should_send_email, mark_email_as_sent
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from helper_database import extract_store_listings_huurstunt

import logging

In [30]:
logger = logging.getLogger(__name__)
handler = logging.StreamHandler()
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)
logger.propagate = False

load_dotenv()

HUURSTUNT_USERNAME = os.getenv('HUURSTUNT_USERNAME')
HUURSTUNT_PASSWORD = os.getenv('HUURSTUNT_PASSWORD')

DATABASE_NAME = "listings.db"


chrome_options = webdriver.ChromeOptions()
chrome_options.debugger_address = "127.0.0.1:9222"
driver = webdriver.Chrome(options=chrome_options)

In [20]:
driver.get("https://www.huurstunt.nl/")
set_filters_huurstunt(driver, 100, 1500, 'Amsterdam')

In [59]:
listings = extract_store_listings_huurstunt(driver)
for listing in listings:
    listing_id = listing['id']  # assuming each listing is a dictionary with an 'id' key
    if should_send_email    (listing_id):
        print(f"Email hasn't been sent for listing ID: {listing_id}")
    else:
        print(f"Email has already been sent for listing ID: {listing_id}")


Email hasn't been sent for listing ID: 1
Email hasn't been sent for listing ID: 2
Email hasn't been sent for listing ID: 3
Email hasn't been sent for listing ID: 4
Email hasn't been sent for listing ID: 5
Email hasn't been sent for listing ID: 6
Email hasn't been sent for listing ID: 7
Email hasn't been sent for listing ID: 9
Email hasn't been sent for listing ID: 10
Email hasn't been sent for listing ID: 11
Email hasn't been sent for listing ID: 12
Email hasn't been sent for listing ID: 13
Email hasn't been sent for listing ID: 14
Email hasn't been sent for listing ID: 15
Email hasn't been sent for listing ID: 16
Email hasn't been sent for listing ID: 17
Email hasn't been sent for listing ID: 18
Email hasn't been sent for listing ID: 19
Email hasn't been sent for listing ID: 20
Email hasn't been sent for listing ID: 21


In [60]:
# Step 1: Extract listings
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC

try:
    listings = extract_store_listings_huurstunt(driver)
    logger.info(f'Extracted {len(listings)} listings.')
except Exception as e:
    logger.error(f"Error during listing extraction: {e}")

# Step 2: Process each listing
for listing in listings:
    listing_id = listing['id']  # Use the 'id' from the extracted data

    

    # Check if an email should be sent for this listing
    try:
        if should_send_email(listing_id):
            
            # Navigate to the specific URL of the listing
            driver.get(listing['specific_url'])
            
            wait = WebDriverWait(driver, 10)
            email_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a.agent__info__mail__show[href^='mailto:']")))

            # Extract owner email from the href attribute
            owner_email = email_element.get_attribute('href').replace('mailto:', '')
            print(owner_email)
                    
            # Send the notification email
            # send_notification_to_owner(listing, address_selector, email_selector, listing['specific_url'])
            
            # # Mark the email as sent in the database
            # mark_email_as_sent(listing_id)
    except Exception as e:
        logger.error(f"Error processing listing with ID {listing_id}: {e}")


2023-11-06 21:41:57,996 - INFO - Extracted 20 listings.


KeyboardInterrupt: 

In [67]:
# Step 1: Extract listings

driver.get("https://www.huurstunt.nl/studio/huren/amsterdam/100-1500/")
wait = WebDriverWait(driver, 2)
wait 

<selenium.webdriver.support.wait.WebDriverWait (session="a05ade9b32738a9ff001e99bb9ff0490")>

In [68]:

try:
    listings = extract_store_listings_huurstunt(driver)
    logger.info(f'Extracted {len(listings)} listings.')
except Exception as e:
    logger.error(f"Error during listing extraction: {e}")

# Step 2: Process each listing
for listing in listings[:4]:
    listing_id = listing['id']  # Use the 'id' from the extracted data    
    try:
        if should_send_email(listing_id):
            logger.info(f"Email hasn't been sent for listing ID: {listing_id}")
            
            driver.get(listing['specific_url'])            
            owner_email = driver.execute_script('return document.querySelector("span[data-select-copy=\'1\']").innerText;')

            print(owner_email)
            
    except NoSuchElementException:
        print('in culo')
    
    except Exception as e:
        print('no email found')
        pass

2023-11-06 21:43:08,500 - INFO - Extracted 20 listings.
2023-11-06 21:43:08,518 - INFO - Email hasn't been sent for listing ID: 1
2023-11-06 21:43:08,986 - INFO - Email hasn't been sent for listing ID: 2


no email found


2023-11-06 21:43:09,603 - INFO - Email hasn't been sent for listing ID: 3


info.nl@plazaresidentservices.com


2023-11-06 21:43:10,212 - INFO - Email hasn't been sent for listing ID: 4


no email found
info.nl@plazaresidentservices.com


In [37]:

try:
    listings = extract_store_listings_huurstunt(driver)
    logger.info(f'Extracted {len(listings)} listings.')
except Exception as e:
    logger.error(f"Error during listing extraction: {e}")

# Step 2: Process each listing
for listing in listings[:4]:
    listing_id = listing['id']  # Use the 'id' from the extracted data
    logger.info(f"Processing listing with ID {listing_id}")
    
    # Check if an email should be sent for this listing
    try:
         if should_send_email(listing_id):
            logger.info(f"Email hasn't been sent for listing ID: {listing_id}")
            
            driver.get(listing['specific_url'])            
            owner_email = driver.execute_script('return document.querySelector("span[data-select-copy=\'1\']").innerText;')

            # Print the extracted email for verification
            print(f"Extracted Email for {listing['address']}: {owner_email}")
            
    except NoSuchElementException:
        logging.error(f"Unable to locate email for listing with ID {listing_id}")
        # Send a notification to your OUTLOOK_EMAIL
        print('MAKE THE FUNCTION TO SEND THE EMAIL TO YOURSELF')
        # send_notification_to_my_email(listing['specific_url'])  # Assuming you have a method to send an email to your address
    except Exception as e:
        logging.error(f"No email founded for listing with ID {listing_id}: {e}")

            
    # Commented out the actual sending of the email. Uncomment this when you're ready to send.
    # """
    # # Send the notification email
    # send_notification_to_owner(listing, address_selector, email_selector, listing['specific_url'])
    
    # # Mark the email as sent in the database
    # mark_email_as_sent(listing_id)
    #         """


2023-11-06 21:28:01,164 - INFO - Extracted 20 listings.
2023-11-06 21:28:01,164 - INFO - Extracted 20 listings.
2023-11-06 21:28:01,164 - INFO - Extracted 20 listings.
2023-11-06 21:28:01,167 - INFO - Processing listing with ID 1
2023-11-06 21:28:01,167 - INFO - Processing listing with ID 1
2023-11-06 21:28:01,167 - INFO - Processing listing with ID 1
2023-11-06 21:28:01,171 - INFO - Email hasn't been sent for listing ID: 1
2023-11-06 21:28:01,171 - INFO - Email hasn't been sent for listing ID: 1
2023-11-06 21:28:01,171 - INFO - Email hasn't been sent for listing ID: 1


got the specific url:  https://www.huurstunt.nl/studio/huren/in/amsterdam/dennenrodepad/Q8Hdg


2023-11-06 21:28:03,871 - ERROR - Error processing listing with ID 1: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7C79E8EF2+54786]
	(No symbol) [0x00007FF7C7955612]
	(No symbol) [0x00007FF7C780A64B]
	(No symbol) [0x00007FF7C784B79C]
	(No symbol) [0x00007FF7C784B91C]
	(No symbol) [0x00007FF7C7886D87]
	(No symbol) [0x00007FF7C786BEAF]
	(No symbol) [0x00007FF7C7884D02]
	(No symbol) [0x00007FF7C786BC43]
	(No symbol) [0x00007FF7C7840941]
	(No symbol) [0x00007FF7C7841B84]
	GetHandleVerifier [0x00007FF7C7D37F52+3524194]
	GetHandleVerifier [0x00007FF7C7D8D800+3874576]
	GetHandleVerifier [0x00007FF7C7D85D7F+3843215]
	GetHandleVerifier [0x00007FF7C7A85086+694166]
	(No symbol) [0x00007FF7C7960A88]
	(No symbol) [0x00007FF7C795CA94]
	(No symbol) [0x00007FF7C795CBC2]
	(No symbol) [0x00007FF7C794CC83]
	BaseThreadInitThunk [0x00007FF8D109257D+29]
	RtlUserThreadStart [0x00007FF8D2D2AA78+40]

2023-11-06 21:28:03,871 - ERROR - Error processing listing with ID 1: Message: 
Stacktrace:
	GetHandleVer

got the specific url:  https://www.huurstunt.nl/studio/huren/in/duivendrecht/lunaweg/Q8ARl


2023-11-06 21:28:06,595 - ERROR - Error processing listing with ID 2: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7C79E8EF2+54786]
	(No symbol) [0x00007FF7C7955612]
	(No symbol) [0x00007FF7C780A64B]
	(No symbol) [0x00007FF7C784B79C]
	(No symbol) [0x00007FF7C784B91C]
	(No symbol) [0x00007FF7C7886D87]
	(No symbol) [0x00007FF7C786BEAF]
	(No symbol) [0x00007FF7C7884D02]
	(No symbol) [0x00007FF7C786BC43]
	(No symbol) [0x00007FF7C7840941]
	(No symbol) [0x00007FF7C7841B84]
	GetHandleVerifier [0x00007FF7C7D37F52+3524194]
	GetHandleVerifier [0x00007FF7C7D8D800+3874576]
	GetHandleVerifier [0x00007FF7C7D85D7F+3843215]
	GetHandleVerifier [0x00007FF7C7A85086+694166]
	(No symbol) [0x00007FF7C7960A88]
	(No symbol) [0x00007FF7C795CA94]
	(No symbol) [0x00007FF7C795CBC2]
	(No symbol) [0x00007FF7C794CC83]
	BaseThreadInitThunk [0x00007FF8D109257D+29]
	RtlUserThreadStart [0x00007FF8D2D2AA78+40]

2023-11-06 21:28:06,595 - ERROR - Error processing listing with ID 2: Message: 
Stacktrace:
	GetHandleVer

got the specific url:  https://www.huurstunt.nl/studio/huren/in/amsterdam/holendrechtstraat/Q87FY


2023-11-06 21:28:09,100 - ERROR - Error processing listing with ID 3: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7C79E8EF2+54786]
	(No symbol) [0x00007FF7C7955612]
	(No symbol) [0x00007FF7C780A64B]
	(No symbol) [0x00007FF7C784B79C]
	(No symbol) [0x00007FF7C784B91C]
	(No symbol) [0x00007FF7C7886D87]
	(No symbol) [0x00007FF7C786BEAF]
	(No symbol) [0x00007FF7C7884D02]
	(No symbol) [0x00007FF7C786BC43]
	(No symbol) [0x00007FF7C7840941]
	(No symbol) [0x00007FF7C7841B84]
	GetHandleVerifier [0x00007FF7C7D37F52+3524194]
	GetHandleVerifier [0x00007FF7C7D8D800+3874576]
	GetHandleVerifier [0x00007FF7C7D85D7F+3843215]
	GetHandleVerifier [0x00007FF7C7A85086+694166]
	(No symbol) [0x00007FF7C7960A88]
	(No symbol) [0x00007FF7C795CA94]
	(No symbol) [0x00007FF7C795CBC2]
	(No symbol) [0x00007FF7C794CC83]
	BaseThreadInitThunk [0x00007FF8D109257D+29]
	RtlUserThreadStart [0x00007FF8D2D2AA78+40]

2023-11-06 21:28:09,100 - ERROR - Error processing listing with ID 3: Message: 
Stacktrace:
	GetHandleVer

got the specific url:  https://www.huurstunt.nl/studio/huren/in/amsterdam/rijswijkstraat/Q82SG


2023-11-06 21:28:11,948 - ERROR - Error processing listing with ID 4: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7C79E8EF2+54786]
	(No symbol) [0x00007FF7C7955612]
	(No symbol) [0x00007FF7C780A64B]
	(No symbol) [0x00007FF7C784B79C]
	(No symbol) [0x00007FF7C784B91C]
	(No symbol) [0x00007FF7C7886D87]
	(No symbol) [0x00007FF7C786BEAF]
	(No symbol) [0x00007FF7C7884D02]
	(No symbol) [0x00007FF7C786BC43]
	(No symbol) [0x00007FF7C7840941]
	(No symbol) [0x00007FF7C7841B84]
	GetHandleVerifier [0x00007FF7C7D37F52+3524194]
	GetHandleVerifier [0x00007FF7C7D8D800+3874576]
	GetHandleVerifier [0x00007FF7C7D85D7F+3843215]
	GetHandleVerifier [0x00007FF7C7A85086+694166]
	(No symbol) [0x00007FF7C7960A88]
	(No symbol) [0x00007FF7C795CA94]
	(No symbol) [0x00007FF7C795CBC2]
	(No symbol) [0x00007FF7C794CC83]
	BaseThreadInitThunk [0x00007FF8D109257D+29]
	RtlUserThreadStart [0x00007FF8D2D2AA78+40]

2023-11-06 21:28:11,948 - ERROR - Error processing listing with ID 4: Message: 
Stacktrace:
	GetHandleVer

In [25]:
for listing in listings[:4]:
    listing_id = listing['id']
    try:
        email = driver.execute_script('return document.querySelector("span[data-select-copy=\'1\']").innerText;')
        # email = email_element.text
        print(f"Extracted Email for {listing_id}: {email}")
    except NoSuchElementException:
        print(f"Error: Couldn't extract email for {listing_id}")

JavascriptException: Message: javascript error: Cannot read properties of null (reading 'innerText')
  (Session info: chrome=118.0.5993.120)
Stacktrace:
	GetHandleVerifier [0x00007FF7C79E8EF2+54786]
	(No symbol) [0x00007FF7C7955612]
	(No symbol) [0x00007FF7C780A64B]
	(No symbol) [0x00007FF7C780F116]
	(No symbol) [0x00007FF7C78114AA]
	(No symbol) [0x00007FF7C7885394]
	(No symbol) [0x00007FF7C786BE6A]
	(No symbol) [0x00007FF7C7884D02]
	(No symbol) [0x00007FF7C786BC43]
	(No symbol) [0x00007FF7C7840941]
	(No symbol) [0x00007FF7C7841B84]
	GetHandleVerifier [0x00007FF7C7D37F52+3524194]
	GetHandleVerifier [0x00007FF7C7D8D800+3874576]
	GetHandleVerifier [0x00007FF7C7D85D7F+3843215]
	GetHandleVerifier [0x00007FF7C7A85086+694166]
	(No symbol) [0x00007FF7C7960A88]
	(No symbol) [0x00007FF7C795CA94]
	(No symbol) [0x00007FF7C795CBC2]
	(No symbol) [0x00007FF7C794CC83]
	BaseThreadInitThunk [0x00007FF8D109257D+29]
	RtlUserThreadStart [0x00007FF8D2D2AA78+40]


In [46]:
email = driver.execute_script('return document.querySelector("span[data-select-copy=\'1\']").innerText;')
email


'info.nl@plazaresidentservices.com'

In [ ]:
import smtplib
from email.message import EmailMessage

def send_notification_email():
    msg = EmailMessage()
    msg.set_content("The email element was not found on the page.")
    msg["Subject"] = "Email Extraction Error!"
    msg["From"] = "your_email_address"
    msg["To"] = "your_outlook_email"

    # Establish a connection to your SMTP server
    with smtplib.SMTP("your_smtp_server", your_smtp_port) as server:
        server.login("your_email_address", "your_email_password")
        server.send_message(msg)
